In [1]:
import pandas as pd
import numpy as np
import sys
from common import commons
home = commons.home
from features_preprocess import get_winid
import os
import re

In [2]:
def nearest_tss(tss,sites_df):
    merged = pd.merge(sites_df,tss,how='outer',on=['chr','coordinate'])
    merged.sort_values(['chr','coordinate'],inplace=True)
    merged.rename(columns={'strand':'before_tss'},inplace=True)
    merged.ix[merged['before_tss'].isnull()==False, 'before_tss'] = merged.ix[merged['before_tss'].isnull()==False,'coordinate']
    merged['after_tss'] = merged['before_tss']
    merged['before_tss'].fillna(method='ffill', inplace=True)
    merged['after_tss'].fillna(method='bfill',inplace=True)
    merged['dist_to_before_tss'] = np.abs(merged['coordinate']-merged['before_tss'])
    merged['dist_to_after_tss'] = np.abs(merged['coordinate']-merged['after_tss'])
    merged['tss'] = None
    before_ix = (merged['dist_to_before_tss'] < merged['dist_to_after_tss']) | (merged['dist_to_after_tss'].isnull())
    merged.ix[before_ix,'tss'] = merged.ix[before_ix,'before_tss']
    after_ix = (merged['dist_to_before_tss'] >= merged['dist_to_after_tss']) | (merged['dist_to_before_tss'].isnull())
    merged.ix[after_ix,'tss'] = merged.ix[after_ix,'after_tss']
    merged['dist_to_nearest_tss'] = np.abs(merged['coordinate']-merged['tss']) 
    merged.drop(['before_tss','after_tss','tss','dist_to_before_tss','dist_to_after_tss'],axis=1,inplace=True)
    merged.dropna(axis=0,subset=['id'],inplace=True)
    return merged

def rename_features(x):   #rename repetitive features
    features = np.array(x.columns)
    features_count = pd.Series(index=x.columns.unique())
    features_count = features_count.fillna(int(0))
    for i,name in enumerate(x.columns):
        if features_count[name] == 0:
            features_count[name] += 1
        else:
            features[i] = name+str(features_count[name])
            features_count[name] += 1
    x.columns = features
    return 

In [3]:
dataset = 'AD_CpG'
with pd.HDFStore(home+'data/'+dataset+'/all_sites_winid','r') as h5s:
    all_sites = h5s['all_sites_winid']
all_sites.reset_index(drop=True,inplace=True)

In [4]:
all_sites

,id,chr,coordinate,beta_sign,pvalue,beta,label,start,winid,end
0,cg09756115,1,879415,NaN,NaN,0.740752,0,879401,4398,879600
1,cg22381068,1,906363,NaN,NaN,0.241956,0,906201,4532,906400
2,cg11211792,1,949634,NaN,NaN,0.779614,0,949601,4749,949800
3,cg04788999,1,949850,NaN,NaN,0.692312,0,949801,4750,950000
4,cg08530610,1,991567,NaN,NaN,0.692247,0,991401,4958,991600
5,cg14144728,1,1101462,NaN,NaN,0.788976,0,1101401,5508,1101600
6,cg13362546,1,1102960,NaN,NaN,0.780087,0,1102801,5515,1103000
7,cg07146103,1,1170069,NaN,NaN,0.798505,0,1170001,5851,1170200
8,cg01937247,1,1229534,NaN,NaN,0.761588,0,1229401,6148,1229600
9,cg03116557,1,1375480,NaN,NaN,0.878502,0,1375401,6878,1375600


In [5]:
feature_dir = home+'data/features/'+dataset+'/'
files = os.listdir(feature_dir)
pattern = '.*all.csv$'
reg = re.compile(pattern)
files = [name for name in files if len(reg.findall(name))>0]




In [6]:
for file in files:    
    feature = pd.read_csv(feature_dir+file)
    print(len(feature.columns))
    all_sites = pd.concat([all_sites,feature],axis=1)

31
267
317
73
80
735
303


In [7]:
rename_features(all_sites)

In [8]:
all_sites.shape

(1556, 1816)

In [9]:
all_sites.drop(['start','end'],axis=1,inplace=True)

In [10]:
additional_features = ['ATAC','CADD','DANN','Eigen','GenoCanyon','RNASeq','WGBS']
#merge with additional features
with pd.HDFStore(feature_dir+'addtional_features','r') as h5s:
    for feature in additional_features:
        feature_frame = h5s[feature]
        all_sites = pd.concat([all_sites,feature_frame],axis=1)
all_sites = all_sites.loc[:,~all_sites.columns.duplicated()]
all_sites['chr'] = all_sites['chr'].astype('i8')

In [14]:
all_sites.ix[:,:1950]

,id,chr,coordinate,beta_sign,pvalue,beta,label,winid,A549,Astrocy,...,ENCFF723ZMR_RNASeq_counts,ENCFF301ROZ_RNASeq_counts,ENCFF888ZFS_RNASeq_counts,ENCFF105THO_RNASeq_counts,ENCFF760IDU_RNASeq_counts,ENCFF624VBI_RNASeq_counts,ENCFF552FTX_RNASeq_counts,ENCFF623UTC_RNASeq_counts,ENCFF535JQR_RNASeq_counts,ENCFF003JVR_WGBS_counts
0,cg23440882,1,875880,0.0238,0.358200,0.030570,0,4380,9,5,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,14.0,0.0,0.045455
1,cg24685837,1,982225,0.0459,0.169100,0.604002,0,4912,5,1,...,1540.0,12.0,0.0,147.0,0.0,93.0,36.0,179.0,0.0,2.538462
2,cg02494066,1,983386,0.0421,0.238000,0.587702,0,4917,4,3,...,369.0,5.0,0.0,69.0,0.0,48.0,28.0,74.0,0.0,1.888889
3,cg20685419,1,1007730,0.0436,0.229800,0.443294,0,5039,4,4,...,0.0,0.0,0.0,0.0,2.0,9.0,0.0,1.0,1.0,2.416667
4,cg00305285,1,1017115,0.0044,0.926000,0.722598,0,5086,3,5,...,0.0,0.0,0.0,26.0,1.0,3.0,3.0,3.0,1.0,2.846154
5,cg15207999,1,1021210,0.0071,0.810100,0.802394,0,5107,3,3,...,225.0,23.0,1.0,173.0,11.0,80.0,36.0,27.0,7.0,9.181818
6,cg05929553,1,1086836,0.0199,0.577400,0.835167,0,5435,2,2,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,5.0,3.000000
7,cg07115976,1,1155731,-0.1745,0.000138,0.906688,-1,5779,8,2,...,37.0,2.0,0.0,59.0,2.0,19.0,7.0,33.0,1.0,6.571429
8,cg00211609,1,1178039,0.0233,0.451700,0.402827,0,5891,5,0,...,0.0,0.0,0.0,2.0,0.0,16.0,0.0,10.0,1.0,1.666667
9,cg02136596,1,1384930,0.0116,0.787200,0.776592,0,6925,0,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [11]:
#nearest tss distance    
chrs = all_sites['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)
tss.sort_values(['chr','coordinate'],inplace=True)
all_sites = nearest_tss(tss,all_sites)


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [12]:
with pd.HDFStore(home+'data/'+dataset+'/all_features','w') as h5s:
    h5s['all_features'] = all_sites